In [1]:
import pandas as pd

import os


path = "./dll_analysis/malware"
filename = "fixed_rules.csv"
file_to_scan = os.path.join(path,filename)
df1 = pd.read_csv(file_to_scan)

temp_families = open(path+"/most_frequent_families.csv", "r").readlines()

In [11]:
print(temp_families)
most_frequent_families = set()
"""NOTE: check the format when moving to EXE. This was for DLL only."""
for i in range(len(temp_families)):
    if(i==0):
        continue
    most_frequent_families.add(temp_families[i].strip('\n').split(",")[2])
""" For EXEs, or, in general, for any file as a list of families one per line:

for i in range(len(temp_families)):
    most_frequent_families.append(temp_families[i].strip('\n'))
"""
print(most_frequent_families)

[',first_seen:month,Family,count\n', '1,2016-10,epack,2\n', '2,2016-10,locky,275\n', '3,2016-10,razy,25\n', '4,2016-10,zusy,2\n', '6,2016-11,agentb,4\n', '7,2016-11,locky,655\n', '8,2016-11,midie,5\n', '9,2016-11,ranserkd,12\n', '10,2016-11,razy,6\n', '11,2016-11,waldek,5\n', '12,2016-12,locky,2\n', '13,2017-01,locky,1\n']
set(['razy', 'locky', 'agentb', 'midie', 'epack', 'ranserkd', 'waldek', 'zusy'])


In [12]:
import pefile
import ssdeep

m1 = df1.loc[:,["Exe name", "Family", "first_seen", "imphash"]]
m1["first_seen:month"] = pd.to_datetime(m1["first_seen"]).dt.to_period("M")

In [13]:
del m1["first_seen"]

In [14]:
m1.head()

,Exe name,Family,imphash,first_seen:month
0,05c0f641bc943f8c1b6c1eddafebc2ee107676b3b2c57a...,locky,cba475024652ed6788f8d77ae1a30155,2016-11
1,1524b86c0e00ac5b93af3db19265a79ec269ef67573802...,locky,a23d5ed45b5bb2d45988b4d790bb2fb7,2016-10
2,d82b557fc694192fd928fbf916a979c0838eec6d42f334...,razy,3736395e43e13030dde5ba3bc3da56d5,2016-11
3,33fbae9c05b6e46aa3360bd20d2f39216b91b6c167595a...,locky,3736395e43e13030dde5ba3bc3da56d5,2016-11
4,58114b0f4eb890f498b96eed084c2f341f711dbf9fc941...,locky,9cd36be5889827aef4f69402aa78a2ba,2016-11


In [15]:
df_dict={el: pd.DataFrame() for el in most_frequent_families}

In [22]:
import ssdeep
for family in most_frequent_families:
    print(family)
    mf = m1[:][m1.Family == family]
    most_frequent_months = mf.groupby(mf["first_seen:month"]).size().reset_index(name="count")
    
    most_frequent_months = most_frequent_months[most_frequent_months["count"]>5]["first_seen:month"]
    df_month ={ el: pd.DataFrame() for el in most_frequent_months.values}
    for month in most_frequent_months.values:
        mm = mf[:][mf["first_seen:month"]==month]
        print(mm.shape)
        cp = mm.merge(mm, how = 'outer', on = "Family")
        """ Performing cartesian product of the dataset with itself
    ( it will be useful to use the apply function over the same row of the dataframe).
    Each row will now contain two samples of the same family 
    (too complex and, probably, useless to perform it over all the samples, independently on the family)"""
        cp["IAT equality"] = cp.apply(lambda x: int(x["imphash_x"]==x["imphash_y"]), axis=1)
        df_month[month] = cp
        #print(cp)
    df_dict[family] = df_month


razy
(25, 4)
(6, 4)
locky
(275, 4)
(655, 4)
agentb
midie
epack
ranserkd
(12, 4)
waldek
zusy


In [17]:
import os
os.mkdir(path+"/heatmap/iat_heatmap/monthly_analysis")

In [26]:
import matplotlib.pyplot as plt
for family in most_frequent_families:
    os.mkdir(path+"/heatmap/iat_heatmap/monthly_analysis/"+family)
    for month, df_month in df_dict[family].items():
        print(family, month)
        df_month = df_month.pivot(index= "Exe name_x", columns="Exe name_y", values="IAT equality")
        ax = plt.pcolor(df_month)
        ax.get_figure().savefig(path+"/heatmap/iat_heatmap/monthly_analysis/"+family+"/"+str(month)+"_heatmap", dpi=800)
        plt.clf()
        df_month.to_csv(path+"/heatmap/iat_heatmap/monthly_analysis/"+family+"/"+str(month)+"_heatmap.csv")

('razy', Period('2016-11', 'M'))
('razy', Period('2016-10', 'M'))
('locky', Period('2016-11', 'M'))
('locky', Period('2016-10', 'M'))
('ranserkd', Period('2016-11', 'M'))


In [19]:
print(path)

./dll_analysis/malware
